## 5. Google places API

In [83]:
import pandas as pd
import os, time
import geopandas as gpd
import geopy.distance

In [2]:
df_cal_tech = pd.read_csv('/Users/mikhailgasanov/Downloads/ChargePoint Data CY20Q4.csv')

/Users/mikhailgasanov/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 1.Cluster points in groups

In [180]:
cols_to_plot = ['Latitude', 'Longitude',
                'Total Duration (hh:mm:ss)',       
                'Charging Time (hh:mm:ss)', 
                'Energy (kWh)', 'Station Name']

gdf = gpd.GeoDataFrame(data=df_cal_tech[cols_to_plot], 
                       geometry=gpd.points_from_xy(x=df_cal_tech['Latitude'], 
                                                   y=df_cal_tech['Longitude']))

In [32]:
grouped = gdf.groupby(['Longitude', 'Latitude'], as_index=False).mean()
centroids = grouped.round(3).groupby(['Longitude', 'Latitude'], as_index=False).mean()
centroids = gpd.GeoDataFrame(data=centroids, 
                       geometry=gpd.points_from_xy(x=centroids['Latitude'],
                                                   y=centroids['Longitude']))

In [194]:
import plotly.express as px
import base64
import plotly.graph_objects as go

token = open('../.mapbox_token').read()


fig_1 = px.scatter_mapbox(centroids, lat='Latitude', lon='Longitude',
                         color_continuous_scale='blues',
                          zoom=14, height=600)

fig_1.update_layout(mapbox_style="satellite",mapbox_accesstoken=token )

fig_1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig_1.update_traces(marker=dict(size=16))

## 2. Request Google Places API

In [ ]:
def getDistance(coords_1:tuple, coords_2:tuple) -> float:
    return geopy.distance.geodesic(coords_1, coords_2).m

def getBuildings(response: dict) -> pd.DataFrame:
    buildings_lat = [i['geometry']['location']['lat'] for i in response['results']]
    buildings_lon = [i['geometry']['location']['lng'] for i in response['results']]
    buildings = [i['types'][0] for i in response['results']]
    buildings_id = [i['reference'] for i in response['results']]
    df_buildings = pd.DataFrame(data=[buildings, buildings_lat, buildings_lon, buildings_id]).T
    df_buildings.columns = ['category', 'latitude_bld', 'longitude_bld', 'id_google']
    return df_buildings

def getNearby(latitude: float, longitude:float, distance:int=1000) -> pd.DataFrame:
    coordinates = (latitude, longitude)
    response = client.places_nearby(
        location = coordinates,
        radius = distance)
    next_page_token = response.get('next_page_token')

    df_buildings = getBuildings(response=response)

    print('Download', end=' ')
    while next_page_token:
        print('.', end='')
        time.sleep(2)
        response = client.places_nearby(
        location = coordinates,
        radius = distance,
        page_token = next_page_token)
        _df = getBuildings(response=response)
        df_buildings = pd.concat([df_buildings, _df])
        next_page_token = response.get('next_page_token')
    print('Finished')
    df_buildings.loc[:, 'Latitude'] = latitude
    df_buildings.loc[:, 'Longitude'] = longitude
    return df_buildings


In [36]:
token_google = open('/Users/mikhailgasanov/.google_token', 'r').read()

In [38]:
import googlemaps
client = googlemaps.Client(key = token_google)


In [193]:
(len(centroids))

17

In [94]:
spatial_features = pd.DataFrame()
for i in range(len(centroids)):
    row = centroids.iloc[i,:]
    latitude = row['Latitude']
    longitude = row['Longitude']
    df = getNearby(latitude=latitude, longitude=longitude, distance=1000)
    spatial_features = pd.concat([spatial_features, df])

Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished
Download ..Finished


In [102]:
spatial_features.loc[:, 'distance'] = spatial_features.apply(lambda x:
                                                             getDistance(coords_1=(x.latitude_bld, 
                                                                                   x.longitude_bld),
                                                                        coords_2=(x.latitude, 
                                                                                   x.longitude)),axis=1)


In [129]:
spatial_features['id_google'].unique()

316

## 3. Get Popularity

In [105]:
import populartimes

In [134]:
def getPopularity(id_google: str)->pd.DataFrame:
    resp = populartimes.get_id(token_google, id_google)
    headers = []
    data = []
    if 'populartimes' in resp.keys():
        for i in resp['populartimes']:
            headers.extend([i['name']+ '_'+ str(h) for h in range(1,25)])
            data.extend(i['data'])
        df = pd.DataFrame(data).T
        df.columns = headers
        df.loc[:, 'id_google'] = id_google

        return df 
    else: 
        return None

In [140]:
popularity_df = pd.DataFrame()
i = 0
for id_google in spatial_features['id_google'].unique():
    print(i)
    i=i+1
    _df = getPopularity(id_google=id_google)
    if _df is None:
        continue
    else:
        popularity_df = pd.concat([popularity_df, _df])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [141]:
popularity_df.head()

,Monday_1,Monday_2,Monday_3,Monday_4,Monday_5,Monday_6,Monday_7,Monday_8,Monday_9,Monday_10,...,Sunday_16,Sunday_17,Sunday_18,Sunday_19,Sunday_20,Sunday_21,Sunday_22,Sunday_23,Sunday_24,id_google
0,1,0,0,0,1,9,30,55,60,42,...,16,19,22,23,19,15,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4
0,0,0,0,0,0,0,0,0,0,0,...,94,84,67,0,0,0,0,0,0,ChIJOXaCcTS7j4AR1Hjo8LCVWAc
0,0,0,0,0,0,0,0,0,0,0,...,0,0,29,42,52,48,0,0,0,ChIJwWcdxzi7j4ARSPpV37nhr58
0,0,0,0,0,0,0,0,0,0,0,...,28,34,37,37,33,26,0,0,0,ChIJpf2l3zi7j4ARq9jTYIKdfb0
0,0,0,0,0,0,0,0,0,0,0,...,46,43,48,54,51,0,0,0,0,ChIJdwmtjjm7j4ARuuLA95wq7wQ


## 4. Merge with initial dataset

In [144]:
merged_buildings = popularity_df.merge(spatial_features, on='id_google', how='left')

In [145]:
merged_buildings

,Monday_1,Monday_2,Monday_3,Monday_4,Monday_5,Monday_6,Monday_7,Monday_8,Monday_9,Monday_10,...,Sunday_22,Sunday_23,Sunday_24,id_google,category,latitude_bld,longitude_bld,latitude,longitude,distance
0,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.443,-122.163,1772.477477
1,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.444,-122.163,1783.665043
2,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.446,-122.162,1740.644646
3,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.447,-122.162,1773.003050
4,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.444,-122.160,1520.901437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,0,0,0,0,0,0,0,0,9,27,...,0,0,0,ChIJs2q9C226j4ARohCdKz3cBH8,meal_takeaway,37.418736,-122.109067,37.422,-122.112,445.688756
313,0,0,0,0,0,0,0,0,52,61,...,0,0,0,ChIJL6KAkXK6j4ARywJAtaxPmt4,school,37.417618,-122.10935,37.422,-122.114,637.105602
314,0,0,0,0,0,0,0,0,52,61,...,0,0,0,ChIJL6KAkXK6j4ARywJAtaxPmt4,school,37.417618,-122.10935,37.422,-122.113,583.843193
315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,ChIJBVNwaxK6j4ARUPhWmCDHXYk,restaurant,37.42035,-122.102235,37.422,-122.113,970.377547


In [163]:

gdf.loc[:, 'Latitude_src'] = gdf.loc[:, 'Latitude']
gdf.loc[:, 'Longitude_src'] = gdf.loc[:, 'Longitude']
gdf.loc[:, 'latitude'] = gdf.loc[:, 'Latitude'].round(3)
gdf.loc[:, 'longitude'] = gdf.loc[:, 'Longitude'].round(3)

final = gdf.merge(merged_buildings, on=['longitude', 'latitude'], how='left')

In [176]:
merged_buildings

,Monday_1,Monday_2,Monday_3,Monday_4,Monday_5,Monday_6,Monday_7,Monday_8,Monday_9,Monday_10,...,Sunday_22,Sunday_23,Sunday_24,id_google,category,latitude_bld,longitude_bld,latitude,longitude,distance
0,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.443,-122.163,1772.477477
1,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.444,-122.163,1783.665043
2,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.446,-122.162,1740.644646
3,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.447,-122.162,1773.003050
4,1,0,0,0,1,9,30,55,60,42,...,11,8,4,ChIJORy6nXuwj4ARz3b1NVL1Hw4,locality,37.441883,-122.143019,37.444,-122.160,1520.901437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,0,0,0,0,0,0,0,0,9,27,...,0,0,0,ChIJs2q9C226j4ARohCdKz3cBH8,meal_takeaway,37.418736,-122.109067,37.422,-122.112,445.688756
313,0,0,0,0,0,0,0,0,52,61,...,0,0,0,ChIJL6KAkXK6j4ARywJAtaxPmt4,school,37.417618,-122.10935,37.422,-122.114,637.105602
314,0,0,0,0,0,0,0,0,52,61,...,0,0,0,ChIJL6KAkXK6j4ARywJAtaxPmt4,school,37.417618,-122.10935,37.422,-122.113,583.843193
315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,ChIJBVNwaxK6j4ARUPhWmCDHXYk,restaurant,37.42035,-122.102235,37.422,-122.113,970.377547


In [181]:
gdf.loc[:, 'latitude'] = gdf.loc[:, 'Latitude'].round(3)
gdf.loc[:, 'longitude'] = gdf.loc[:, 'Longitude'].round(3)

In [209]:
final = merged_buildings.merge(centroids[['longitude','latitude','cluster']],
                               on=['longitude', 'latitude'])

In [199]:
centroids.loc[:, 'cluster'] = range(len(centroids))
centroids.loc[:, 'cluster'] = centroids.loc[:, 'cluster'] + 1

In [206]:
centroids.columns = ['longitude', 'latitude', 'Energy (kWh)', 'geometry', 'cluster']

In [211]:
final.to_csv('../Datasets/spatial_populartimes.csv')

In [214]:
df_cal_tech.loc[:, 'latitude'] = df_cal_tech.loc[:, 'Latitude'].round(3)
df_cal_tech.loc[:, 'longitude'] = df_cal_tech.loc[:, 'Longitude'].round(3)

df_cal_tech = df_cal_tech.merge(centroids[['longitude','latitude','cluster']], on=['longitude','latitude'])

In [216]:
df_cal_tech.to_csv('../Datasets/ChargePoint Data CY20Q4 SPATAIL.csv')

## End 